# Ivan Zepeda
C0883949

Word normalization => convert word to root form

import nltk
from nltk.corpus import wordnet

w1 = wordnet.synset('ship.n.01')
w1 = wordnet.synset('ship.n.01')


### Word Lengthening:
a word might onoly have same letter repeated twice: book, cool, collapse

In [2]:
import re
def remove_lenghtening(text):
    pat= re.compile(r'(.)\1{2,}')
    return pat.sub(r'\1\1', text)
print(remove_lenghtening("booook"))
print(remove_lenghtening("booookk"))
print(remove_lenghtening("booookkkk"))
print(remove_lenghtening("booookkokookkk"))




book
bookk
bookk
bookkokookk


# Spell Correction

NLTK Does not have a solution

In [3]:
# pip install autocorrect
from autocorrect import Speller
spell = Speller(lang='en')
print(spell('mussage'))
print(spell('survice'))
print(spell('hte'))
print(spell('caar'))
print(spell('saaw'))
print(spell('wsa'))
print(spell('looosee'))
print(spell('caar'))
print(spell('caaar'))
print(spell('caaaar'))
print(spell('caaaaar'))


# the model, doesnt work well with lenghtening
# it can also work with suggestions

message
service
the
car
saw
was
loose
car
car
aaaaaa
aaaaaa


## MEthod 2
### PAttern

In [4]:
#pip install pattern
from pattern.en import suggest

In [7]:
print(suggest('mussage'))
print(suggest('survice'))
print(suggest('hte'))
print(suggest('caar'))
print(suggest('caaar'))
print(suggest('caaaar'))
print(suggest('caaaaar'))



[('message', 0.6216216216216216), ('massage', 0.3783783783783784)]
[('service', 0.9253112033195021), ('survive', 0.07468879668049792)]
[('the', 0.8653201565642368), ('he', 0.13408515883485067), ('ate', 0.00022706139307570876), ('hate', 0.0002162489457863893), ('hue', 0.00012974936747183358), ('te', 1.0812447289319465e-05), ('htm', 1.0812447289319465e-05)]
[('car', 0.5), ('czar', 0.42857142857142855), ('saar', 0.07142857142857142)]
[('clear', 0.4251824817518248), ('chair', 0.24635036496350365), ('canal', 0.13138686131386862), ('clair', 0.04927007299270073), ('makar', 0.03467153284671533), ('cigar', 0.02737226277372263), ('caesar', 0.01824817518248175), ('car', 0.012773722627737226), ('czar', 0.010948905109489052), ('bazaar', 0.009124087591240875), ('cava', 0.005474452554744526), ('afar', 0.005474452554744526), ('clara', 0.0036496350364963502), ('cedar', 0.0036496350364963502), ('cabal', 0.0036496350364963502), ('saar', 0.0018248175182481751), ('cavae', 0.0018248175182481751), ('cater', 

## WORDNET


built in data to help train models  
has data classification  


**CLassification**: metrics: Accuracy, F1_score, recall, ROC, KS,  
**Regression:** R2 (percentage of variation on Y are influenced by X. I fit is high is a good sign )

In [8]:
import nltk
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/ivan/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [9]:
len(movie_reviews.words())

1583820

In [10]:
movie_reviews.categories()

['neg', 'pos']

In [11]:
movie_reviews.fileids()[:5]

['neg/cv000_29416.txt',
 'neg/cv001_19502.txt',
 'neg/cv002_17424.txt',
 'neg/cv003_12683.txt',
 'neg/cv004_12641.txt']

# TExt preprocessing

In [12]:
# Remove the punctuations
text = " ".join(movie_reviews.words())
import string

text_filtered = text.translate(str.maketrans("","", string.punctuation))

In [13]:
from nltk.tokenize import word_tokenize
text_filtered_tokens= word_tokenize(text_filtered)
print(len(text_filtered_tokens)) #words

1337085


In [14]:
print(len(text_filtered)) #letters

7559896


In [15]:
text_filtered[:10]

'plot  two '

In [16]:
# then remove the stopwords and then make all words lowercase

In [17]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
tokens = word_tokenize(text_filtered)
word_filtered = [w.lower() for w in tokens if w not in stopwords]

#### We can use FreqDist() funciton on NLTK to have a dictionary of frequency of appearance of words ina text



In [18]:
# Create a dictionary, 
counter_dict = nltk.FreqDist(word_filtered)
print(counter_dict.most_common(15))

[('film', 9519), ('one', 5853), ('movie', 5774), ('like', 3690), ('even', 2565), ('good', 2411), ('time', 2411), ('story', 2170), ('would', 2110), ('much', 2050), ('character', 2020), ('also', 1967), ('get', 1949), ('two', 1912), ('well', 1906)]


In [19]:
len(word_filtered)

708475

In [20]:
import time
tic=time.time()
print(len(set(word_filtered)))
print("{:.6f}".format(time.time()-tic),"secs from set len")
tic=time.time()
print(len(counter_dict))
print("{:.6f}".format(time.time()-tic),"secs, from filtered_word")


39295
0.071936 secs from set len
39295
0.000082 secs, from filtered_word


In [21]:
docs = [(list(movie_reviews.words(fileid)), category)
       for category in movie_reviews.categories()
       for fileid in movie_reviews.fileids(category)]

# in every category (pos or neg), take entire file ids, (each review own id)
# store the word_tokenized version (list of words) for the file id , 
    # and then followed by pos or neg label in one big list




In [22]:
len(docs)

2000

In [23]:
print(docs[0][1])
print(docs[0][0][:10])

neg
['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party']


# Feature Extraction
creata a list of 3k most frequent words in docs



In [24]:
word_features = [w[0] for w in counter_dict.most_common(3000)]

def search_features(doc):
    words = set(doc)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [32]:
featureset = [(search_features(doc), category) for (doc, category) in docs]
# featureset[:10]

## Train SET

In [33]:
training_set = featureset[:1600]
testing_set = featureset[1600:]

# Demonstrate the training of the text classification model (Naive Bayes)

In [34]:
# Training
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [35]:
# Analyze
print("Accuracy is:", nltk.classify.accuracy(classifier, testing_set)*100)

Accuracy is: 73.75


## Decision Tree Classifier 

dt_classifier = nltk.DecisionTreeClassifier.train(training_set)
print("Accuracy is:", nltk.classify.accuracy(dt_classifier, testing_set)*100)

## Sklearn Classifier

skl_classifier = nltk.SklearnClassifier.train(training_set)
print("Accuracy is:", nltk.classify.accuracy(skl_classifier, testing_set)*100)